<a href="https://colab.research.google.com/github/MohammadQasim03/Intelligence-Systems-C0559/blob/main/CNN_MINST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np  # Importing numpy library for numerical operations
import keras  # Importing Keras library for building neural networks
from keras.datasets import mnist  # Importing MNIST dataset from Keras
from keras.models import Model  # Importing Model class from Keras
from keras.layers import Dense, Input  # Importing Dense and Input layers from Keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten  # Importing CNN layers from Keras
from keras.preprocessing.image import ImageDataGenerator  # Importing ImageDataGenerator for data augmentation
from keras import backend as k  # Importing backend module from Keras for backend-specific operations
import matplotlib.pyplot as plt  # Importing matplotlib for plotting
from sklearn.metrics import confusion_matrix  # Importing confusion_matrix function from sklearn
import seaborn as sns  # Importing seaborn for visualization

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # Loading MNIST dataset into training and testing sets

img_rows, img_cols = 28, 28  # Setting image dimensions

# Reshape data based on backend (channels-first or channels-last)
if k.image_data_format() == 'channels_first':  # Checking if channels are first
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)  # Reshaping training data
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)  # Reshaping testing data
    inpx = (1, img_rows, img_cols)  # Input shape for channels-first
else:  # If channels are last
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)  # Reshaping training data
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)  # Reshaping testing data
    inpx = (img_rows, img_cols, 1)  # Input shape for channels-last

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255  # Normalizing training data
x_test = x_test.astype('float32') / 255  # Normalizing testing data

# One-hot encode labels
y_train = keras.utils.to_categorical(y_train)  # One-hot encoding training labels
y_test = keras.utils.to_categorical(y_test)  # One-hot encoding testing labels

# Define the CNN model
inpx = Input(shape=inpx)  # Defining input layer
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx)  # Adding first convolutional layer
layer2 = Conv2D(64, (3, 3), activation='relu')(layer1)  # Adding second convolutional layer
layer3 = MaxPooling2D(pool_size=(2, 2))(layer2)  # Adding max pooling layer
layer4 = Dropout(0.25)(layer3)  # Adding dropout layer
layer5 = Flatten()(layer4)  # Flattening the output from convolutional layers
layer6 = Dense(128, activation='relu')(layer5)  # Adding dense layer
layer7 = Dropout(0.5)(layer6)  # Adding dropout layer
layer8 = Dense(10, activation='softmax')(layer7)  # Adding output layer with softmax activation

model = Model([inpx], layer8)  # Creating the model
model.compile(optimizer='adam',  # Compiling the model with Adam optimizer
              loss='categorical_crossentropy',  # Using categorical crossentropy as loss function
              metrics=['accuracy'])  # Using accuracy as evaluation metric

# Data augmentation
datagen = ImageDataGenerator(rotation_range=10,  # Augmenting data with rotation
                             width_shift_range=0.1,  # Augmenting data with width shift
                             height_shift_range=0.1,  # Augmenting data with height shift
                             zoom_range=0.1)  # Augmenting data with zoom
datagen.fit(x_train)  # Fitting the ImageDataGenerator on training data

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=128),  # Training the model with augmented data
                    epochs=30,  # Setting number of epochs
                    validation_data=(x_test, y_test))  # Using validation data during training

# Plot accuracy versus time
plt.plot(history.history['accuracy'], label='Training Accuracy')  # Plotting training accuracy
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  # Plotting validation accuracy
plt.xlabel('Epoch')  # Setting x-label
plt.ylabel('Accuracy')  # Setting y-label
plt.legend()  # Adding legend
plt.show()  # Displaying the plot

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)  # Evaluating the model on test data
print('Loss:', score[0])  # Printing loss
print('Accuracy:', score[1])  # Printing accuracy

# Confusion Matrix
y_pred = model.predict(x_test)  # Predicting labels for test data
y_pred_classes = np.argmax(y_pred, axis=1)  # Getting predicted classes
y_true = np.argmax(y_test, axis=1)  # Getting true classes
conf_matrix = confusion_matrix(y_true, y_pred_classes)  # Computing confusion matrix

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))  # Setting figure size
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))  # Plotting confusion matrix
plt.xlabel('Predicted labels')  # Setting x-label
plt.ylabel('True labels')  # Setting y-label
plt.title('Confusion Matrix')  # Setting title
plt.show()  # Displaying the plot


Epoch 1/30
469/469 [==============================] - 21s 42ms/step - loss: 0.4941 - accuracy: 0.8441 - val_loss: 0.0505 - val_accuracy: 0.9845
Epoch 2/30
469/469 [==============================] - 20s 43ms/step - loss: 0.2053 - accuracy: 0.9391 - val_loss: 0.0380 - val_accuracy: 0.9869
Epoch 3/30
469/469 [==============================] - 21s 44ms/step - loss: 0.1619 - accuracy: 0.9528 - val_loss: 0.0326 - val_accuracy: 0.9897
Epoch 4/30
469/469 [==============================] - 21s 44ms/step - loss: 0.1373 - accuracy: 0.9588 - val_loss: 0.0291 - val_accuracy: 0.9897
Epoch 5/30
469/469 [==============================] - 20s 44ms/step - loss: 0.1224 - accuracy: 0.9639 - val_loss: 0.0329 - val_accuracy: 0.9891
Epoch 6/30
469/469 [==============================] - 19s 41ms/step - loss: 0.1146 - accuracy: 0.9661 - val_loss: 0.0256 - val_accuracy: 0.9909
Epoch 7/30
469/469 [==============================] - 21s 44ms/step - loss: 0.1051 - accuracy: 0.9694 - val_loss: 0.0246 - val_accuracy: